## Librerias a Utilizar
Seccion de libreria que se utilizaran en el proceso de extraccion

In [234]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
from tabulate import tabulate
import re
import time
from datetime import date


# Manejo Web, paginas y webScrapping
# ==============================================================================
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup as bs


# Gráficos
# ==============================================================================
from matplotlib import pyplot as plt


# Configuración warnings
# ==============================================================================\n",
import warnings
warnings.filterwarnings('ignore')

## Funciones 

In [235]:
def ExtraerLink(linkPage,patron):
    lista = []
    for tag in linkPage:
        valor = tag.get('href')
        if(str(valor).find(patron) != -1):
            lista.append(valor)
    df = pd.DataFrame (lista, columns = ['url'])
    df = df.drop_duplicates()
    return df
def leerUrl(pagina):    
    soup = bs(urllib.request.urlopen(pagina).read().decode())
    #print(str(soup) )
    time.sleep(3)
  
    return  soup

## Manejo de Datos y sitios a considerar


In [236]:
# Instantiate the webdriver with the executable location of MS Edge
# Provide the full location of the path to recognise correctly
PATH = 'App\msedgedriver.exe'
edgeBrowser = webdriver.Edge(PATH)

# This is the step for minimize browser window
edgeBrowser.minimize_window()

In [237]:
# Browser will get navigated to the given URL
edgeBrowser.get('https://www.linkedin.com/jobs/search/?keywords=Data%20Scientist&location=Chile&locationId=&geoId=104621616&f_TPR=r86400&position=1&pageNum=0')

time.sleep(3)

# This is the step for minimize browser window
edgeBrowser.minimize_window()

## Rescata las paginas a Scrapear


In [238]:
linkedin_soup = bs(edgeBrowser.page_source.encode("utf-8"), "html")
#linkedin_soup.prettify()

## busqueda de pagina en los campos < A >
Se busca con el patron de link a paginas con la oferta.

In [239]:
patron = '/jobs/view/'
df = ExtraerLink(linkedin_soup('a'),patron)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 26
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     27 non-null     object
dtypes: object(1)
memory usage: 432.0+ bytes


In [240]:
df.head()

,url
0,https://cl.linkedin.com/jobs/view/data-scienti...
1,https://cl.linkedin.com/jobs/view/data-scienti...
2,https://cl.linkedin.com/jobs/view/senior-data-...
3,https://cl.linkedin.com/jobs/view/programador-...
4,https://cl.linkedin.com/jobs/view/react-softwa...


In [241]:
columns = ["url", "contenido"]
dffinal = pd.DataFrame(columns=columns)

for i in range(len(df)-1):
    link = str(df.iloc[i]['url'])
    #print(link)
    linkedin_soup1 = leerUrl(link.split('?')[0]) 
    #print(linkedin_soup1)
    if str(linkedin_soup1) == "Not Found":
        break
 
    parametro = 'show-more-less-html__markup'
    links_divs = linkedin_soup1.findAll("div", {"class": parametro})
    links_divs = str(links_divs)
    dffinal = dffinal.append(
                {
                    "url": link,
                    "contenido": links_divs,
                },
                ignore_index=True,
            )

In [257]:
# Traza
#with pd.option_context('display.max_colwidth', None):
#    display(dffinal)

dffinal.to_parquet("output/df_"+date.today().strftime("%d%m%Y")+'.parquet')
